In [2]:
!pip install tiledb

In [3]:
import tiledb
import numpy as np
from tqdm import tqdm
import torch

print(tiledb.__version__)

0.32.5


In [7]:
import tiledb
import numpy as np
import os
import shutil

# Define the dataset parameters for 4000 timestamps
T = 4000  # Timestamps
X = 855  # Spatial X dimension
Y = 1215  # Spatial Y dimension
r = 50
# **Define ε here:**
epsilon = 1e-2
dtype = np.float32  # Data type of the `.DAT` file

# Paths
dat_file_path = r"Redsea_t2_gan\Redsea_t2_4k_gan.DAT"
tiledb_array_path = "arrayD"
tiledb_ur_path = "arrayUr"
tiledb_sr_path = "arraySr"
tiledb_vr_path = "arrayVr"
tiledb_correction_path = "arrayE_intermediate"
tiledb_quantized_correction_path = "arrayE"

def creating_quantized_E():
    domain = tiledb.Domain(
        tiledb.Dim(name="time", domain=(0, T-1), tile=4000, dtype=np.int32),
        tiledb.Dim(name="x", domain=(0, X-1), tile=1, dtype=np.int32),
        tiledb.Dim(name="y", domain=(0, Y-1), tile=1215, dtype=np.int32),
    )
    schema = tiledb.ArraySchema(
        domain=domain,
        sparse=True,
        attrs=[tiledb.Attr(name="correction", dtype=np.uint16)]
    )
    tiledb.SparseArray.create(tiledb_quantized_correction_path, schema) # creating E
    
    with tiledb.open(tiledb_correction_path, mode="r") as E, tiledb.open(tiledb_quantized_correction_path, mode="w") as quant_E:
        # Query the data with coordinates
        result = E[:]

        # Access the coordinates and values
        time_coords = result["time"]  # Replace "time" with your dimension name
        x_coords = result["x"]        # Replace "x" with your dimension name
        y_coords = result["y"]        # Replace "y" with your dimension name
        values_of_E = result["correction"]  # Replace "correction" with your attribute name
        min_error = values_of_E.min()
        max_error = values_of_E.max()
        
        quantized_E = (np.round((values_of_E - min_error) / (max_error - min_error) * 65535)).astype(np.uint16)
        quant_E[time_coords, x_coords, y_coords] = quantized_E

    return min_error, max_error

# Utility function to delete TileDB paths if they exist
def delete_tiledb_paths(paths):
    for path in paths:
        if os.path.exists(path):
            shutil.rmtree(path)
            print(f"Deleted existing TileDB array at: {path}")

# Utility function to get the size of a TileDB array
def get_array_size(path):
    return sum(os.path.getsize(os.path.join(dirpath, f)) for dirpath, _, files in os.walk(path) for f in files)

# Step 1: Create TileDB Array from `.DAT` File
def create_tiledb_array(dat_file_path, tiledb_array_path):
    
    domain = tiledb.Domain(
        tiledb.Dim(name="time", domain=(0, T-1), tile=500, dtype=np.int32),
        tiledb.Dim(name="x", domain=(0, X-1), tile=1, dtype=np.int32),
        tiledb.Dim(name="y", domain=(0, Y-1), tile=1215, dtype=np.int32),
    )
    schema = tiledb.ArraySchema(
        domain=domain,
        sparse=False,
        attrs=[tiledb.Attr(name="temperature", dtype=dtype)]
    )
##########################################################################################################################################################
    chunksize = 855*1215*500 # loads 500 timesteps at a time
    min_value, max_value = 1000, 0

    tiledb.DenseArray.create(tiledb_array_path, schema)
    with tiledb.DenseArray(tiledb_array_path, mode="w") as array:
        total = 0
        for i in range(T//500):
            data = np.fromfile(dat_file_path, dtype=dtype, count=chunksize, offset=chunksize*i*4, ).reshape(500, X, Y) #loading the whole data (no creativity)
            total += np.sum(data)

            min_value = min(min_value, data.min())
            max_value = max(max_value, data.max())

            #calculate data average

            array[i*500:(i+1)*500,:,:] = {"temperature": data}
    print(f"TileDB array created at {tiledb_array_path}")
    average = total / (T*X*Y)

    return min_value, max_value, average
#############################################################################################################################################################

# Step 2: Perform SVD and Store Components
def perform_svd_and_store(tiledb_array_path, tiledb_ur_path, tiledb_sr_path, tiledb_vr_path, r, average):

    tiledb.DenseArray.create(tiledb_ur_path, tiledb.ArraySchema(
        domain=tiledb.Domain(
            tiledb.Dim(name="num_svd", domain=(0, X-1), tile=1, dtype=np.int32), # (how many SVD's, U_r.shape[0], U_r.shape[1])
            tiledb.Dim(name="rows", domain=(0, T-1), tile=T, dtype=np.int32), # (how many SVD's, U_r.shape[0], U_r.shape[1])
            tiledb.Dim(name="cols", domain=(0, r-1), tile=r, dtype=np.int32)
        ),
        attrs=[tiledb.Attr(name="U_r", dtype=np.float32)]
    ))
    tiledb.DenseArray.create(tiledb_sr_path, tiledb.ArraySchema(
        domain=tiledb.Domain(
            tiledb.Dim(name="num_svd", domain=(0, X-1), tile=1, dtype=np.int32), # (how many SVD's, S_r.shape[0])
            tiledb.Dim(name="components", domain=(0, r-1), tile=r, dtype=np.int32) # (how many SVD's, S_r.shape[0])
        ),
        attrs=[tiledb.Attr(name="S_r", dtype=np.float32)]
    ))
    tiledb.DenseArray.create(tiledb_vr_path, tiledb.ArraySchema(
        domain=tiledb.Domain(
            tiledb.Dim(name="num_svd", domain=(0, X-1), tile=1, dtype=np.int32), # (how many SVD's, V_r.shape[0], V_r.shape[1])
            tiledb.Dim(name="rows", domain=(0, r-1), tile=r, dtype=np.int32), # (how many SVD's, V_r.shape[0], V_r.shape[1])
            tiledb.Dim(name="cols", domain=(0, Y-1), tile=Y, dtype=np.int32) #Txr, rxr, rxY
        ),
        attrs=[tiledb.Attr(name="V_r", dtype=np.float32)]
    ))

    with tiledb.DenseArray(tiledb_array_path, mode="r") as array, tiledb.DenseArray(tiledb_ur_path, mode="w") as array_ur, \
        tiledb.DenseArray(tiledb_sr_path, mode="w") as array_sr, tiledb.DenseArray(tiledb_vr_path, mode="w") as array_vr:

        for x in tqdm(range(855)): # how many svd's
        
            data = torch.tensor(array[:, x, :]["temperature"], dtype=torch.float32).to("cuda:0") - average

            # data_flat = data.reshape(data.shape[0], -1)
            U, S, Vt = torch.linalg.svd(data, full_matrices=False)
            U_r, S_r, V_r = U[:, :r], S[:r], Vt[:r, :]

            # store components in TileDB arrays
            U_r = U_r.cpu().numpy()
            S_r = S_r.cpu().numpy()
            V_r = V_r.cpu().numpy()

            array_ur[x,:,:] = {"U_r": U_r}
            array_sr[x, :] = {"S_r": S_r}
            array_vr[x,:,:] = {"V_r": V_r}

    print(f"SVD components stored in TileDB arrays: {tiledb_ur_path}, {tiledb_sr_path}, {tiledb_vr_path}")

# Step 3: Reconstruct Matrix
def reconstruct_matrix(U_r, S_r, V_r, average): # need to loop reconstruct_matrix for each x
    S_r_diag = np.diag(S_r)
    return np.dot(U_r, np.dot(S_r_diag, V_r)) + average

# Step 4: Compute Correction Matrix
def compute_correction(D, D_reconstructed, vRange, epsilon):
    error = np.abs(D - D_reconstructed)
    correction_needed = (error / vRange) > epsilon
    correction = np.where(correction_needed, D - D_reconstructed, 0)

    return correction

# Step 5: Store Correction Matrix
def store_correction_matrix(correction, tiledb_correction_path, x):

    rows, columns = np.nonzero(correction)

    dim_len = [x for i in range(len(rows))]

    with tiledb.open(tiledb_correction_path, mode="w") as array:
        array[rows, dim_len, columns] = correction[rows, columns] #mapping every element which is nonzero to a 3D coordinate (x, y, z). For first pass: (x, y, z) = (0, rows, columns)

# Step 6: Calculate Compression Ratio
def calculate_compression_ratio(tiledb_array_path, tiledb_ur_path, tiledb_sr_path, tiledb_vr_path, tiledb_correction_path):
    size_D = get_array_size(tiledb_array_path)
    size_Ur = get_array_size(tiledb_ur_path)
    size_Sr = get_array_size(tiledb_sr_path)
    size_Vr = get_array_size(tiledb_vr_path)
    size_E = get_array_size(tiledb_correction_path)
    compression_ratio = size_D / (size_Ur + size_Sr + size_Vr + size_E)
    print("size_E: ", size_E/10**6, "MB")
    print("size_Ur + size_Sr + size_Vr: ", (size_Ur + size_Sr + size_Vr)/10**6, "MB")
    print(f"Final Compression Ratio (ρ): {compression_ratio:.2f}")
    return compression_ratio



### Deleting and Creating

In [8]:
# Delete all existing TileDB arrays
delete_tiledb_paths([tiledb_array_path, tiledb_ur_path, tiledb_sr_path, tiledb_vr_path, tiledb_correction_path, tiledb_quantized_correction_path])

min_value, max_value, average = create_tiledb_array(dat_file_path, tiledb_array_path)  # Step 1
vRange = max_value - min_value # using to calculate error bound
print("Average of D: ", average)
print("created D")

FileNotFoundError: [Errno 2] No such file or directory: 'Redsea_t2_gan\\Redsea_t2_4k_gan.DAT'

### Performing SVD

In [9]:
perform_svd_and_store(tiledb_array_path, tiledb_ur_path, tiledb_sr_path, tiledb_vr_path, r, average)  # Step 2
print("finished svd")


NameError: name 'average' is not defined

In [10]:
with tiledb.DenseArray(tiledb_array_path, mode="r") as array_d, tiledb.DenseArray(tiledb_ur_path, mode="r") as array_ur, \
    tiledb.DenseArray(tiledb_sr_path, mode="r") as array_sr, tiledb.DenseArray(tiledb_vr_path, mode="r") as array_vr:

    domain = tiledb.Domain(
        tiledb.Dim(name="time", domain=(0, T-1), tile=4000, dtype=np.int32),
        tiledb.Dim(name="x", domain=(0, X-1), tile=1, dtype=np.int32),
        tiledb.Dim(name="y", domain=(0, Y-1), tile=1215, dtype=np.int32),
    )
    schema = tiledb.ArraySchema(
        domain=domain,
        sparse=True,
        attrs=[tiledb.Attr(name="correction", dtype=np.float32)]
    )
    tiledb.SparseArray.create(tiledb_correction_path, schema) # creating E

    for x in tqdm(range(855)): #loading the svd components for each x in D and D
        
        U_r = array_ur[x,:,:]["U_r"]
        S_r = array_sr[x, :]["S_r"]
        V_r = array_vr[x,:,:]["V_r"]
        D = array_d[:, x, :]["temperature"]
        
        D_prime = reconstruct_matrix(U_r, S_r, V_r, average)
        vRange = max_value - min_value
        correction = compute_correction(D, D_prime, vRange, epsilon)
        store_correction_matrix(correction, tiledb_correction_path, x)
    

TileDBError: [TileDB::Array] Error: Cannot open array; Array does not exist.

In [116]:
min_error, max_error = creating_quantized_E()
calculate_compression_ratio(tiledb_array_path, tiledb_ur_path, tiledb_sr_path, tiledb_vr_path, tiledb_quantized_correction_path)

size_E:  700.261962 MB
size_Ur + size_Sr + size_Vr:  903.940484 MB
Final Compression Ratio (ρ): 10.36


10.363107480263746

In [6]:
with tiledb.DenseArray(tiledb_array_path, mode="r") as array_d, tiledb.DenseArray(tiledb_ur_path, mode="r") as array_ur, \
    tiledb.DenseArray(tiledb_sr_path, mode="r") as array_sr, tiledb.DenseArray(tiledb_vr_path, mode="r") as array_vr, \
    tiledb.open(tiledb_quantized_correction_path) as array_e:

    for i in range(855):

        print(i)

        U_r = array_ur[i,:,:]["U_r"]
        S_r = array_sr[i, :]["S_r"]
        V_r = array_vr[i,:,:]["V_r"]
        D = array_d[:, i, :]["temperature"]
        
        S_r_diag = np.diag(S_r)
        D_prime = np.dot(U_r, np.dot(S_r_diag, V_r)) + average

        errors = array_e[:,i,:]

        dequantized = errors["correction"]

        dequantized = dequantized.astype(np.float32) / (2**16 - 1) * (max_error - min_error) + min_error

        rows, cols = errors["time"], errors["y"]

        for row, col, value in zip(rows, cols, dequantized):

            D_prime[row, col] += value

        # now its d_prime_prime

        distances = np.abs(D - D_prime)

        print(np.max(distances))

        # assert np.all((distances / vRange) <= epsilon)


print("Passed test!")


TileDBError: [TileDB::Array] Error: Cannot open array; Array does not exist.